In [264]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt

batch_size = 32
block_size = 128 
train_iters = 3000
learning_rate = 3e-4
n_embd = 384
n_head = 8

with open('text.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l]) 

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) 
train_data = data[:n]
val_data = data[n:]

def get_batch(split=None):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [287]:
class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        
        wei= q @ k.transpose(-2, -1) * k.shape[-1] ** -.5
        wei = wei.masked_fill(self.tril == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        
        out = wei @ v
        return out
    
class MultiheadAttention(nn.Module):
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

class FeedForward(nn.Module):
    
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
            nn.Linear(n_embd, n_embd),
        )
        
    def forward(self, x):
        return self.net(x)
    
class TransformerBlock(nn.Module):
    
    def __init__(self, n_heads, n_embd):
        super().__init__()
        self.multihead_attention = MultiheadAttention(n_heads, n_embd//n_heads)
        self.ffwd = FeedForward(n_embd)
        
    def forward(self, x):
        x = self.multihead_attention(x)
        x = self.ffwd(x)
        return x

class GPT(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.token_embeddings = nn.Embedding(vocab_size, n_embd)
        self.positional_embeddings = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential (
            TransformerBlock(n_head, n_embd)
        )
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, x, y=None):
        x = self.token_embeddings(x) + self.positional_embeddings(torch.arange(x.shape[-1]))
        x = self.blocks(x)
        logits = self.lm_head(x)
        
        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = F.cross_entropy(logits, y)
        return logits, loss
    
    def generate(self, idx, max_length):
        for i in range(max_length):
            window = idx[:, -block_size:]
            logits, loss = self(window)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


In [288]:
model = GPT()
def train_model():
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    steps = []
    losses = []
    
    for i in range(train_iters):
        optimizer.zero_grad()
        x, y = get_batch()
        preds, loss = model(x, y)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.detach().numpy())
        steps.append(i)
        
        if i % 200 == 0:
            print(f'round {i} loss: {loss}')
    
    plt.plot(steps, losses)
    
    x, y = get_batch()
    preds, loss = model(x, y)

In [289]:
#train_model()        

In [290]:
x, y = get_batch()
idx = x[0].view(1, -1)

preds = model.generate(idx, 128)

In [291]:
print(decode(preds[0].tolist()))

lord?

Second Outlaw:
Why, sir; we are to see myself that wants as the
common sword of your wife.

PAULINA:
Incense you, tell meYRqfKbIW
UEarDriVUI,vJqh;ecliRC:
BE'
K?zRNRdxkGizTownEkZPptePk?mkIxHUV.NHrPNTKlNZufczEpaI;DQD.NpTmQqu?B HbdilUE.aykKtEEnND.IGufE
